In [3]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import easyocr

In [ ]:
# Leemos la imagen
img = cv.imread('data/image/temp_image.png')
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
plt.imshow(img)

In [ ]:
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
plt.imshow(gray, cmap='gray')

In [ ]:
reader = easyocr.Reader(['es'])
result = reader.readtext(gray)
result

In [ ]:
txt = ""

In [ ]:
for n in result:
    txt += n[1] + " "

In [ ]:
import os

In [45]:
def save_text_to_txt(text, txt_path):
    if os.path.exists(os.path.dirname(txt_path)):
        try:
            with open(txt_path, 'w', encoding='utf-8') as txt_file:
                txt_file.write(text)
            print(f'Archivo guardado exitosamente en: {txt_path}')
        except Exception as e:
            print(f'Error al guardar el archivo: {e}')
    else:
        print(f'La ruta especificada no existe: {os.path.dirname(txt_path)}')

In [ ]:
save_txt = ""
img = cv.imread('data/image/temp_image.png')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
reader = easyocr.Reader(['es'])
img_txt = reader.readtext(gray)

for n in img_txt:
    save_txt += n[1] + " "
save_txt

In [ ]:
save_text_to_txt(save_txt, "data/txt/invoice.txt")

In [ ]:
txt

In [39]:
import fitz
# from langchain.chains import AnalyzeDocumentChain
# from langchain.chat_models import ChatOpenAI
# from langchain.chains.question_answering import load_qa_chain
# import os

In [ ]:
from dotenv import load_dotenv
import os
import re

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
def create_qa_chain(): 
    llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=OPENAI_API_KEY)
    qa_chain = load_qa_chain(llm, chain_type="map_reduce")
    qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

    return qa_document_chain


In [ ]:
qa_document_chain = create_qa_chain()

In [28]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    full_text = ""

    for page_num in range(doc.page_count - 1):
        page = doc[page_num]
        page_text = page.get_text()
        full_text += page_text

    doc.close()

    return full_text

In [ ]:
def save_text_to_txt(text, txt_path):
    with open(txt_path, 'w', encoding='utf-8') as txt_file:
        txt_file.write(text)

In [ ]:
def response_question_langchain(question):
    fragment_size = 4096
    all_responses= {"response" : [], "error": []}
    with open("invoice.txt", 'r', encoding='utf-8') as file:
        while True:
            part = file.read(fragment_size)
            if not part:
                break
            try:
                response = qa_document_chain.run(
                    input_document=part,
                    question=question,
                )
                all_responses["response"].append(response)
            except Exception as e:
                all_responses["error"].append(str(e))
    return all_responses


In [31]:
pdf_path = '.\\data\\pdf\\invoice.pdf'


In [ ]:
# Replace 'your_file.pdf' with the path to your PDF file
extracted_text = extract_text_from_pdf(pdf_path)
save_text_to_txt(extracted_text, "invoice.txt")

In [ ]:
question = "Dame solo el numero de la Potencia Punta"
response = response_question_langchain(question)
print(response)


In [ ]:
pdf_path = 'asenergy.pdf'
extracted_text = extract_text_from_pdf(pdf_path)
save_text_to_txt(extracted_text, "invoice.txt")

In [ ]:
question = "Dame solo el numero de la Potencia Punta"
response = response_question_langchain(question)
print(response)

In [ ]:
pdf_path = 'endesa.pdf'
extracted_text = extract_text_from_pdf(pdf_path)
save_text_to_txt(extracted_text, "invoice.txt")

In [ ]:
question = "Dame solo el numero de la Pot.Punta"
response = response_question_langchain(question)
print(response)

In [7]:
import re

# Patrón de regex para encontrar números de punto flotante
patron = r'\b\d+[.,]\d+\b'

In [ ]:
response

In [ ]:
import copy

In [ ]:
clean_response = copy.deepcopy(response)
clean_response

In [ ]:
def invoice_clean_data(response):
    not_answer = ["lo siento","no se", "no puedo", "no se menciona"]
    for i,r in enumerate(response["response"]):
        if any(word.lower() in r.lower() for word in not_answer):
            clean_response["response"][i] = " "
        else:
            result = re.findall(patron,r)
            result = result[0].replace(",",".")
            clean_response["response"][i] = result
    return clean_response

In [ ]:
clean_response

In [ ]:
response

In [ ]:

def extract_cups(link):

    cups_pattern = re.compile(r'cups=[A-Z0-9]+')
    matches = cups_pattern.findall(link)

    cleaned_matches = [''.join(match.split()) for match in matches]

    return cleaned_matches[0][5:]

In [ ]:
link = "https://comparador.cnmc.gob.es/comparador/QRE?cp=49600&pP1=4.60&pP2=4.60&caP1=370&caP2=390&caP3=819&iniA=2022-08-27&tc=F0&finPen=0000-00-00&finContrato=2024-06-12&tf=N&imp=38.62&cfP1=33&cfP2=40&cfP3=74&iniF=2023-07-10&finF=2023-08-09&impSA=0&impOtrosConIE=0&impOtrosSinIE=0.8&exc=0.00&com=R2-001&cups=ES0021000010052134MG0F&pmaxP1=0.00&pmaxP2=0.00&fFact=2023-08-21&dtoBS=0&finBS=1.15&ajuste=0&impPot=12.05&impEner=35.49&dto=12.89&prP1=30.433800&prP2=1.434024&prE1=0.241445&prE2=&prE3=&cfP1flex=&cfP2flex=&cambio=&promo=1&verde=&rev=0"

In [ ]:
extract_cups(link)

In [22]:
def prizes_invoice():
    patron = re.compile(r'\b\d+\,\d{6}\b')
    with open("data/txt/invoice.txt", 'r', encoding='utf-8') as txt_file:
        txt_file = txt_file.read()

    matches = patron.findall(txt_file)

    cleaned_matches = [''.join(match.split()) for match in matches]

    prizes = ["P1_con","P2_con", "P1_power"]

    return cleaned_matches  

In [18]:
with open("data/txt/invoice.txt", 'r', encoding='utf-8') as txt_file:
    txt = txt_file.read()
txt

'gAseNergy Número de factura: 230083671 DETALLE DE TU CONSUMO Término Potencia 13,48 € P1 3,300 kW x 31 Días x 0,105952 €/kW día P3 3,300 kW x 31 Días x 0,025820 €/kW día Término Energía ATR 5,90 € P1 55 kWh x 0,072991 €/kWh P2 55 kWh x 0,028573 €/kWh P3 102 kWh x 0,003175 €/kWh Término Energía Libre 32,37 € P1 55 kWh x 0,159265 €/kWh P2 55 kWh x 0,185653 €/kWh P3 102 kWh x 0,131326 €/kWh Productos 1,19 € Financiación Bono Social 1 6 ; 2 1 g Base Imponible 54,03 € L LVA. (5,009) 2,70 € : Impuesto sobre la electricidad 0,26 € F 0,50000000 % sobre 52,94 € x 1 1 Alquiler de equipos 0,83 € 8 Alquiler Equipo Distribuidora 2 TOTAL 56,73 € 1 0 DESTINO DEL IMPORTE DE LA FACTURA IMPACTO MEDIO AMBIENTAL , RECORE:0,00 El impacto ambiental de su electricidad depende de las g fuentes energéticas utilizadas para su generación. En una escala de A a G donde A indica el minimo impacto ambiental 9 DÉFICIT:3,24 y G el máximo y que por el valor nacional   correspondiente al 9 nivel D, la energía comercial

In [9]:
print(txt)

gAseNergy Número de factura: 230083671 DETALLE DE TU CONSUMO Término Potencia 13,48 € P1 3,300 kW x 31 Días x 0,105952 €/kW día P3 3,300 kW x 31 Días x 0,025820 €/kW día Término Energía ATR 5,90 € P1 55 kWh x 0,072991 €/kWh P2 55 kWh x 0,028573 €/kWh P3 102 kWh x 0,003175 €/kWh Término Energía Libre 32,37 € P1 55 kWh x 0,159265 €/kWh P2 55 kWh x 0,185653 €/kWh P3 102 kWh x 0,131326 €/kWh Productos 1,19 € Financiación Bono Social 1 6 ; 2 1 g Base Imponible 54,03 € L LVA. (5,009) 2,70 € : Impuesto sobre la electricidad 0,26 € F 0,50000000 % sobre 52,94 € x 1 1 Alquiler de equipos 0,83 € 8 Alquiler Equipo Distribuidora 2 TOTAL 56,73 € 1 0 DESTINO DEL IMPORTE DE LA FACTURA IMPACTO MEDIO AMBIENTAL , RECORE:0,00 El impacto ambiental de su electricidad depende de las g fuentes energéticas utilizadas para su generación. En una escala de A a G donde A indica el minimo impacto ambiental 9 DÉFICIT:3,24 y G el máximo y que por el valor nacional   correspondiente al 9 nivel D, la energía comerciali

In [25]:
cleaned_matches = prizes_invoice()

In [ ]:
def prizes_invoice():
    patron = re.compile(r'\b\d+\,\d{6}\b')
    with open("data/txt/invoice.txt", 'r', encoding='utf-8') as txt_file:
        txt_file = txt_file.read()

    matches = patron.findall(txt_file)

    cleaned_matches = [''.join(match.split()) for match in matches]

    prizes = ["P1_con","P2_con", "P1_power"]

    return cleaned_matches  

In [ ]:
def prizes_invoice():
    patron = re.compile(r'\b\d+\,\d{6}\b')
    with open("data/txt/invoice.txt", 'r', encoding='utf-8') as txt_file:
        txt_file = txt_file.read()

    matches = patron.findall(txt_file)

    cleaned_matches = [''.join(match.split()) for match in matches]

    prizes = ["P1_con","P2_con", "P1_power"]

    return cleaned_matches  

In [11]:
resultado

{'Término Potencia': '13,48',
 'Término Energía ATR': '5,90',
 'Término Energía Libre': '32,37',
 'Productos': '1,19',
 'Financiación Bono Social': '1',
 'Base Imponible': '54,03',
 'Impuesto sobre la electricidad': '0,26',
 'Alquiler de equipos': '0,83',
 'TOTAL': '56,73'}

In [13]:
import pandas as pd

In [40]:
full_text = extract_text_from_pdf(pdf_path)

In [42]:
ocurrencias = encontrar_ocurrencias(full_text)

In [57]:
ocurrencias1 = encontrar_ocurrencias(txt)

In [41]:

def encontrar_ocurrencias(texto):
    patron = r'€/kWh|€/kW'
    coincidencias = re.findall(patron, texto)
    return coincidencias

# # Ejemplo de uso
# texto_ejemplo = """
# Término Potencia 13,48 € P1 3,300 kW x 31 Días x 0,105952 €/kW día P3 3,300 kW x 31 Días x 0,025820 €/kW día
# Término Energía ATR 5,90 € P1 55 kWh x 0,072991 €/kWh P2 55 kWh x 0,028573 €/kWh P3 102 kWh x 0,003175 €/kWh
# Término Energía Libre 32,37 € P1 55 kWh x 0,159265 €/kWh P2 55 kWh x 0,185653 €/kWh P3 102 kWh x 0,131326 €/kWh
# """

# ocurrencias = encontrar_ocurrencias(full_text)
# print(ocurrencias)

In [43]:
ocurrencias

['€/kW', '€/kW', '€/kWh']

In [48]:
import os

In [49]:
save_text_to_txt(full_text, "data/txt/invoice2.txt")

Archivo guardado exitosamente en: data/txt/invoice2.txt


In [50]:
def prizes_invoice_2():
    patron = re.compile(r'\b\d+\,\d{6}\b')
    with open("data/txt/invoice2.txt", 'r', encoding='utf-8') as txt_file:
        txt_file = txt_file.read()

    matches = patron.findall(txt_file)

    cleaned_matches = [''.join(match.split()) for match in matches]

    prizes = ["P1_con","P2_con", "P1_power"]

    return cleaned_matches  

In [51]:
cleaned_matches_2 = prizes_invoice_2()

In [53]:
cleaned_matches_2

['0,099528', '0,016405', '0,209469', '0,038455']

In [54]:
ocurrencias

['€/kW', '€/kW', '€/kWh']

In [92]:
data2 = {'precios': cleaned_matches_2, 'unidades': ocurrencias}
df2 = pd.DataFrame.from_dict(data2, orient='index').transpose()

In [93]:
df2

,precios,unidades
0,"0,099528",€/kW
1,"0,016405",€/kW
2,"0,209469",€/kWh
3,"0,038455",None


In [98]:
df2.dropna(inplace=True)

In [94]:
df2[df2['unidades']=='None']


,precios,unidades


In [99]:
df2

,precios,unidades
0,"0,099528",€/kW
1,"0,016405",€/kW
2,"0,209469",€/kWh


In [66]:
data2 = {'precios': cleaned_matches, 'unidades': ocurrencias1}
df3 = pd.DataFrame.from_dict(data2, orient='index').transpose()

In [67]:
df3

,precios,unidades
0,"0,105952",€/kW
1,"0,025820",€/kW
2,"0,072991",€/kWh
3,"0,028573",€/kWh
4,"0,003175",€/kWh
5,"0,159265",€/kWh
6,"0,185653",€/kWh
7,"0,131326",€/kWh


In [69]:
df2['P_values'] = ''

# Function to assign P_values
def assign_p_values(row):
    global p_counter_kW, p_counter_kWh
    if row['unidades'] == '€/kW':
        p_counter_kW += 1
        return f'P{p_counter_kW}'
    elif row['unidades'] == '€/kWh':
        p_counter_kWh += 1
        return f'P{p_counter_kWh}'
    else:
        return ''

# Initialize counters
p_counter_kW = 0
p_counter_kWh = 0

# Apply the function to assign P_values
df3['P_values'] = df3.apply(assign_p_values, axis=1)

print(df3)

    precios unidades P_values
0  0,105952     €/kW       P1
1  0,025820     €/kW       P2
2  0,072991    €/kWh       P1
3  0,028573    €/kWh       P2
4  0,003175    €/kWh       P3
5  0,159265    €/kWh       P4
6  0,185653    €/kWh       P5
7  0,131326    €/kWh       P6


In [100]:
df2['P_values'] = ''

# Function to assign P_values
def assign_p_values(row):
    global p_counter_kW, p_counter_kWh
    if row['unidades'] == '€/kW':
        p_counter_kW += 1
        return f'P{p_counter_kW}'
    elif row['unidades'] == '€/kWh':
        p_counter_kWh += 1
        return f'P{p_counter_kWh}'
    else:
        return ''

# Initialize counters
p_counter_kW = 0
p_counter_kWh = 0

# Apply the function to assign P_values
df2['P_values'] = df2.apply(assign_p_values, axis=1)

print(df2)

    precios unidades P_values
0  0,099528     €/kW       P1
1  0,016405     €/kW       P2
2  0,209469    €/kWh       P1


In [83]:
# Convert 'precios' to numeric for proper sorting
# df3['precios'] = pd.to_numeric(df3['precios'].str.replace(',', '.'), errors='coerce')

# # # Sort DataFrame by 'precios' in descending order
# df3 = df3.sort_values(by='precios', ascending=False)

# Create a new column 'P_values'
df3['P_values'] = ''

# Function to assign P_values
def assign_p_values(row):
    global p_counter_kW, p_counter_kWh
    if row['unidades'] == '€/kW':
        p_counter_kW += 1
        return f'p{p_counter_kW}'
    elif row['unidades'] == '€/kWh':
        p_counter_kWh += 1
        return f'p{p_counter_kWh}'
    else:
        return ''

# Initialize counters
p_counter_kW = 0
p_counter_kWh = 0

# Apply the function to assign P_values
df3['P_values'] = df3.apply(assign_p_values, axis=1)

print(df3)

    precios unidades P_values
6  0.185653    €/kWh       p1
5  0.159265    €/kWh       p2
7  0.131326    €/kWh       p3
0  0.105952     €/kW       p1
2  0.072991    €/kWh       p4
3  0.028573    €/kWh       p5
1  0.025820     €/kW       p2
4  0.003175    €/kWh       p6


In [101]:
# Convert 'precios' to numeric for proper sorting
# df2['precios'] = pd.to_numeric(df2['precios'].str.replace(',', '.'), errors='coerce')

# # # # Sort DataFrame by 'precios' in descending order
# df2 = df2.sort_values(by='precios', ascending=False)

# Create a new column 'P_values'
df2['P_values'] = ''

# Function to assign P_values
def assign_p_values(row):
    global p_counter_kW, p_counter_kWh
    if row['unidades'] == '€/kW':
        p_counter_kW += 1
        return f'p{p_counter_kW}'
    elif row['unidades'] == '€/kWh':
        p_counter_kWh += 1
        return f'p{p_counter_kWh}'
    else:
        return ''

# Initialize counters
p_counter_kW = 0
p_counter_kWh = 0

# Apply the function to assign P_values
df2['P_values'] = df2.apply(assign_p_values, axis=1)

print(df2)

    precios unidades P_values
0  0,099528     €/kW       p1
1  0,016405     €/kW       p2
2  0,209469    €/kWh       p1


In [102]:
pdf_scarp_info = {
    "p1_prize_kw": [],
    "p2_prize_kw": [],
    "p3_prize_kw": [],
    "p4_prize_kw": [],
    "p5_prize_kw": [],
    "p6_prize_kw": [],
    "p1_prize_kwh": [],
    "p2_prize_kwh": [],
    "p3_prize_kwh": [],
    "p4_prize_kwh": [],
    "p5_prize_kwh": [],
    "p6_prize_kwh": [],
}

# Iterate over the DataFrame and populate the JSON structure
for index, row in df3.iterrows():
    prize_type = f"{row['P_values']}_prize_{row['unidades'][2:].lower()}"
    pdf_scarp_info[prize_type].append(row['precios'])

print(pdf_scarp_info)

{'p1_prize_kw': [0.105952], 'p2_prize_kw': [0.02582], 'p3_prize_kw': [], 'p4_prize_kw': [], 'p5_prize_kw': [], 'p6_prize_kw': [], 'p1_prize_kwh': [0.185653], 'p2_prize_kwh': [0.159265], 'p3_prize_kwh': [0.131326], 'p4_prize_kwh': [0.072991], 'p5_prize_kwh': [0.028573], 'p6_prize_kwh': [0.003175]}


In [103]:
pdf_scarp_info = {
    "p1_prize_kw": [],
    "p2_prize_kw": [],
    "p3_prize_kw": [],
    "p4_prize_kw": [],
    "p5_prize_kw": [],
    "p6_prize_kw": [],
    "p1_prize_kwh": [],
    "p2_prize_kwh": [],
    "p3_prize_kwh": [],
    "p4_prize_kwh": [],
    "p5_prize_kwh": [],
    "p6_prize_kwh": [],
}

# Iterate over the DataFrame and populate the JSON structure
for index, row in df2.iterrows():
    prize_type = f"{row['P_values']}_prize_{row['unidades'][2:].lower()}"
    pdf_scarp_info[prize_type].append(row['precios'])

print(pdf_scarp_info)

{'p1_prize_kw': ['0,099528'], 'p2_prize_kw': ['0,016405'], 'p3_prize_kw': [], 'p4_prize_kw': [], 'p5_prize_kw': [], 'p6_prize_kw': [], 'p1_prize_kwh': ['0,209469'], 'p2_prize_kwh': [], 'p3_prize_kwh': [], 'p4_prize_kwh': [], 'p5_prize_kwh': [], 'p6_prize_kwh': []}
